# Train the model

In [1]:
from ultralytics import YOLO
import os
import boto3

In [2]:
! tar -xzf dataset-full.tar.gz

In [3]:
EPOCHS=int(os.environ.get("epochs", 50))
IMGSZ=640
NB_FROZEN_LAYER=0
DATA_CONFIG_PATH = "utils/data.yaml"

EXPORT_BUCKET = "model-registry"
MODEL_PATH = "dev/model.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

In [4]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [5]:
client.download_file(EXPORT_BUCKET, "default/best.pt", "base-model.pt")

In [6]:
model = YOLO('base-model.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ, freeze=NB_FROZEN_LAYER)

New https://pypi.org/project/ultralytics/8.3.3 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.9.16 torch-2.3.1+cu121 CPU (AMD EPYC 7571)
engine/trainer: task=detect, mode=train, model=base-model.pt, data=utils/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=0, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

100%|██████████| 755k/755k [00:00<00:00, 90.0MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /opt/app-root/src/activity-notebooks/dataset/labels/train... 79 images, 0 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<00:00, 365.50it/s]

train: New cache created: /opt/app-root/src/activity-notebooks/dataset/labels/train.cache


...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      2.468      3.318      1.625         59        640: 100%|██████████| 5/5 [01:31<00:00, 18.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.71s/it]


                   all         15         48      0.742      0.458      0.612      0.246

1 epochs completed in 0.028 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.9.16 torch-2.3.1+cu121 CPU (AMD EPYC 7571)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.23s/it]


                   all         15         48      0.742      0.458      0.612      0.247
            SpeedLimit         11         15      0.733      0.733       0.76      0.358
           DangerAhead         13         33       0.75      0.182      0.463      0.136
Speed: 1.7ms preprocess, 272.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train


In [7]:
model = YOLO('runs/detect/train/weights/best.pt')

In [8]:
model.export(format="onnx")

Ultralytics YOLOv8.2.48 🚀 Python-3.9.16 torch-2.3.1+cu121 CPU (AMD EPYC 7571)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.9 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.2s, saved as 'runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (3.4s)
Results saved to /opt/app-root/src/activity-notebooks/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=utils/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.onnx'

In [9]:
model_path = "runs/detect/train/weights/best.onnx"

In [10]:
client.upload_file(model_path, EXPORT_BUCKET, MODEL_PATH)

In [11]:
client.upload_file('runs/detect/train/weights/best.pt', EXPORT_BUCKET, 'dev/best.pt')